In [ ]:
# Import the random forest package
from sklearn.ensemble import RandomForestClassifier 

# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 100)

# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(training[0::,2::],training[0::,1])

# Take the same decision trees and run it on the test data
output = forest.predict(test)